# Document Classification Project


In [1]:
# set up libraries 
import pandas as pd
import numpy as np
import glob
import os
from bs4 import BeautifulSoup
import requests
import spacy
from spacy.lang.en import English
from collections import Counter
# modelling stuff
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
# plotting stuff
import chart_studio.plotly as py
import plotly.graph_objects as go
import cufflinks as cf
import seaborn as sns
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()


In [2]:
docs = pd.read_csv("C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/Data2.csv")
docs.dropna(inplace=True)
print(docs.iloc[22][0])

https://counteriedreport.com/egyptian-military-kills-77-terrorists-in-northern-sinai-report/


In [4]:
# Get txt files from urls
for i in range(len(docs)):
    try:
        r = requests.get(docs.iloc[i][0], timeout=10)
        print(i, r)
        print(docs.iloc[i][0])
    except:
        print("Problem:", i)
        continue

    if r.ok:
        soup = BeautifulSoup(r.content, 'html.parser')
        with open("data/Reports/" + str(i) + ".txt", 'wt') as f:
            text = ""
            for element in soup.select('p'):
                # Reduce potential overfitting by not including parts of the document that repeat themselves regularly and aren't relevant
                # to the actual text
                if  "View/Print Poster" in element.text:
                    continue
                elif element.text == "If you prefer a different language, you may select one from the full list by navigating to the Select Language menu at the top of this site.":
                    continue
                elif element.text == "Rewards for Justice":
                    continue
                elif element.text == "The most important reasons to stop a terrorist are all around you.":
                    continue
                elif element.text == "Stop a terrorist. Save lives.":
                    continue
                elif element.text == "Up to $10 Million Reward":
                    continue
                elif element.text == "{{region_detail.text}}":
                    continue
                elif element.text == "{{person_detail.text}}":
                    continue
                elif element.text == "{{organization_detail.text}}":
                    continue
                elif element.text == "{{series.description | limitTo:140}}":
                    continue
                elif element.text == "Up to $5 Million Reward":
                    continue
                elif len(element.text.split()) == 1:
                    continue
                text += element.text + "\n"
            try:
                f.write(text)
            except:
                print("Write problem:", i)
                continue
        


0 <Response [200]>
https://issafrica.org/iss-today/time-to-tackle-violence-against-women-in-mali
1 <Response [200]>
http://www.understandingwar.org/backgrounder/russia-review-putins-%E2%80%9Cpeacekeepers%E2%80%9D-will-support-russian-wars
2 <Response [200]>
http://www.understandingwar.org/backgrounder/russian-president-putin-wins-upset-victory-nagorno-karabakh
3 <Response [200]>
http://www.understandingwar.org/backgrounder/warning-intelligence-update-iran-increases-pressure-us-forces-iraq
Write problem: 3
4 <Response [200]>
http://www.understandingwar.org/backgrounder/russia-turkey-competition-escalates-across-theaters
5 <Response [200]>
https://www.middleeasteye.net/news/uk-foreign-aid-cut-impact-middle-east-yemen-syria
6 <Response [200]>
https://www.middleeasteye.net/opinion/israel-palestine-deal-century-biden-end
7 <Response [200]>
https://www.middleeasteye.net/news/israel-saudi-arabia-netanyahu-mbs-secret-meeting-meaning
8 <Response [200]>
https://www.defenseindustrydaily.com/f18-h

In [3]:
# Get data
df = pd.DataFrame(columns=['Text', 'Label'])
main_path = r'C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/'
end_path = ['Analytical_Report', 'Incident_Report', 'Situation_Report']
for end in end_path:
    path = main_path + end
    all_files = glob.glob(os.path.join(path, "*.txt")) 
    # print(all_files[0])
    data = [[f, end] for f in all_files]
    # print(pd.DataFrame(data, columns=['Text', 'Label']))
    df = df.append(pd.DataFrame(data, columns=['Text', 'Label']))
# print(df)

main_path= r'C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/Reports'
files = glob.glob(os.path.join(main_path, '*.txt'))
data2 = [[f, docs.iloc[i][1]] for i, f in enumerate(files)]
df = df.append(pd.DataFrame(data2, columns=['Text', 'Label']))
df.reset_index(inplace = True) 
print(df)

     index                                               Text  \
0        0  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
1        1  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
2        2  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
3        3  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
4        4  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
..     ...                                                ...   
329    172  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
330    173  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
331    174  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
332    175  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
333    176  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   

                 Label  
0    Analytical_Report  
1    Analytical_Report  
2    Analytical_Report  
3    Analytical_Report  
4    Analytical_Report  
..                 ...  
329  Analytical Report  
330  Analytical Rep

In [4]:
# Convert 'label' column to categories
df["Label"] = df["Label"].replace({'Analytical Report' : 'Analytical_Report', 'Incident Report' : 'Incident_Report', 'Profile Report' : 'Profile_Report','Report' : 'Profile_Report', 'Situation Report' : 'Situation_Report'})
df["Label"] = df["Label"].astype('category')
df["Label"]

0      Analytical_Report
1      Analytical_Report
2      Analytical_Report
3      Analytical_Report
4      Analytical_Report
             ...        
329    Analytical_Report
330    Analytical_Report
331    Analytical_Report
332    Analytical_Report
333    Analytical_Report
Name: Label, Length: 334, dtype: category
Categories (4, object): ['Analytical_Report', 'Incident_Report', 'Profile_Report', 'Situation_Report']

In [5]:
# How may documents in each category?
df.groupby(['Label']).count()

,index,Text
Label,,
Analytical_Report,47,47
Incident_Report,88,88
Profile_Report,100,100
Situation_Report,99,99


Now it's time to add some potentially useful features to the dataset, by looking at what we already know about the documents and making some assumptions. Then we can perform EDA to check how relevant these features are before running models using only the most important features.

In [9]:
# Set up lists that will be appended to as we iterate through our df of docs
word_counts, unique_word_counts, vocabs_per_wc, non_stop_per_wc, pos_counts, event_word_count, date_count, time_count, percentage_count, money_count, quantity_count, ordinal_count, cardinal_count = [], [], [], [], [], [], [], [], [], [], [], [], []

# create spacy object
nlp = spacy.load('en_core_web_sm')

# go through each row and read in the text
for idx, row in df.iterrows():
    # read in the text
    try:   
        corpus = open(row[1], encoding="utf8").read()
    except:
        corpus = open(row[1], encoding="latin1").read()
    doc = nlp(corpus)
    list_of_words = [token.text for token in doc if not token.is_punct and token.text != "\n"] # get the words and numbers (not punctiation)

    word_count = len(list_of_words) # Get the word count
    unique_word_count = len(set(list_of_words)) # Get the word count of unique words
    vocab_per_wc = unique_word_count / word_count # Get the proportion of words that are unique
    
    # get a list of words that are not stop words or punctuation
    list_of_non_stop_words = [token.text for token in doc if not token.is_stop and not token.is_punct and token.text != "\n"]
    
    # count how many of each type of part of speech in doc
    pos_count = doc.count_by(spacy.attrs.POS)
    new_pos_count = {}
    for k, v in pos_count.items():
        new_pos_count[doc.vocab[k].text] = v/word_count
    
    date_count.append(len([ent for ent in doc.ents if ent.label_=='DATE'])/word_count)
    time_count.append(len([ent for ent in doc.ents if ent.label_=='TIME'])/word_count)
    percentage_count.append(len([ent for ent in doc.ents if ent.label_=='PERCENT'])/word_count)
    money_count.append(len([ent for ent in doc.ents if ent.label_=='MONEY'])/word_count)
    quantity_count.append(len([ent for ent in doc.ents if ent.label_=='QUANTITY'])/word_count)
    ordinal_count.append(len([ent for ent in doc.ents if ent.label_=='ORDINAL'])/word_count)
    cardinal_count.append(len([ent for ent in doc.ents if ent.label_=='CARDINAL'])/word_count)
    # count the number of numbers in each number range (named entity)
    # count the number of dates (named entity)
    
    word_counts.append(word_count)
    unique_word_counts.append(unique_word_count)
    vocabs_per_wc.append(vocab_per_wc)
    non_stop_per_wc.append(len(set(list_of_non_stop_words))/ word_count) # get proportion of unique non-stop words
    pos_counts.append(new_pos_count)

df['Wordcount'] = word_counts
df['Vocabulary'] = unique_word_counts
df['Vocabulary_per_wc'] = vocabs_per_wc
df['Non-stop_per_wc'] = non_stop_per_wc
df['Dates'] = date_count
df['Time'] = time_count
df['Money'] = money_count
df['Percentages'] = percentage_count
df['Quantity'] = quantity_count
df['Ordinal'] = ordinal_count
df['Cardinal'] = cardinal_count

pos_df = pd.DataFrame(pos_counts)
df = pd.concat([df, pos_df], axis=1)
# print(df['Wordcount'])
# print(df['Vocabulary'])
# print(df["Vocabulary_per_wordcount"])
# print(df.head())

print(df)

     index                                               Text  \
0        0  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
1        1  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
2        2  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
3        3  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
4        4  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
..     ...                                                ...   
329    172  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
330    173  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
331    174  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
332    175  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   
333    176  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   

                 Label  Wordcount  Vocabulary  Vocabulary_per_wc  \
0    Analytical_Report        974         410           0.420945   
1    Analytical_Report       4052        1114           0.274926   
2    Analytical

In [6]:

nlp = spacy.load('en_core_web_sm')
doc1 = nlp("Apple is looking at buying U.K. startup for $1 billion")
doc1.ents

(Apple, U.K., $1 billion)

## Boxplots for each variable in the dataframe, to compare differences in the text between the 4 types of documents.


In [33]:
for i, column in enumerate(df):
    if i > 2:
        px.box(df, x='Label', y=column, title="A boxplot comparing " + column +  " with each document type").show()
    

Discus results here